## Adding operations between audio data files

In [1]:

import os
import glob
from pathlib import Path
import datetime
#import random
import numpy as np
import pandas as pd
from threading import Thread

from microfaune import audio

In [2]:

def mesure_delta(time_start, time_end):
    """ time mesurements
    input a (= datetime.datetime.now())
    input b (= datetime.datetime.now())
    output : seconds, ms, string to display
    """
    c = time_end - time_start
    s = c.seconds
    ms = int(c.microseconds/1000)
    t = str(s) + 's ' + str(ms) + 'ms'
    return s, ms, t


def load_wav_dataonly(wav_filename):
    """ load data without fs """
    fs, data = audio.load_wav(wav_filename)
    return data 


### Basic audio adding files operation 
- audio with same fs size
- audio with different fs size


In [3]:
def add_dataWav1todataWav2(data1, data2, percent1=1, percent2=1):
    """ addition of two wave np array, with weight percentage
    input : 
    data1 first data audio, in flat vector
    data1 first data audio, in flat vector
    percent 1 percentage of data taken
    percent 1 percentage of data added
    output :
    p1*data1 + p2*data2     
    """
    
    if len(data2)==0:
        return data1    
    if len(data2)>len(data1):
        data2 = data2[:len(data1)]
    if len(data2)<len(data1):
        data2 = list(data2) + [1] * (len(data1)-len(data2))
    if percent1==1 and percent2==1:
        list3 = list(map(lambda x, y: x + y, data1, data2))                             
    else:
        list3 = list(map(lambda x, y: (percent1 * x) + (percent2 * y), \
                             data1, data2))
    return np.array(list3, dtype=np.float32) 




##### Test function 

In [4]:
def test_add_dataWav1todataWav2(wave_file='../toy_data/bird1.wav'):
    """ basic tests of wave addition function """
    """ on audio of having the same fs size """
    
    fs, data = audio.load_wav(wave_file)
    data1 = data.copy()
    data2 = np.array([1] * len(data1), dtype=np.float32) 
    
    print('data1==data2')
    print('data1', data1)
    print('data2', data2)
    print()
    
    debut_t = datetime.datetime.now()
    data3 = add_dataWav1todataWav2(data1, data2) 
    s, ms, t = mesure_delta(debut_t, datetime.datetime.now())
    print('Time for data3 = data2 + data1 :', t)
    print('data3', data3)
    print()
    
    debut_t = datetime.datetime.now()
    data3 = add_dataWav1todataWav2(data1, data2, 1, 0.5) 
    s, ms, t = mesure_delta(debut_t, datetime.datetime.now())
    print('Time for data3 = data1 + 50%(data2) :', t)
    print('data3', data3)
    print()
    
    debut_t = datetime.datetime.now()
    data3 = add_dataWav1todataWav2(data1, data2, 0.9, 0) 
    s, ms, t = mesure_delta(debut_t, datetime.datetime.now())
    print('Time for data3 = 90%(data1) + 0%(data2) :', t)
    print('data3', data3)
    print()
    
    data3 = add_dataWav1todataWav2(data1, data2, 1, 0.8) 
    print('data3 = data1 + 80%(data2)')
    print('data3', data3)
    
test_add_dataWav1todataWav2()


data1==data2
data1 [  0.   0.   0. ... 558. 741. 620.]
data2 [1. 1. 1. ... 1. 1. 1.]

Time for data3 = data2 + data1 : 0s 133ms
data3 [  1.   1.   1. ... 559. 742. 621.]

Time for data3 = data1 + 50%(data2) : 1s 562ms
data3 [5.000e-01 5.000e-01 5.000e-01 ... 5.585e+02 7.415e+02 6.205e+02]

Time for data3 = 90%(data1) + 0%(data2) : 1s 563ms
data3 [  0.    0.    0.  ... 502.2 666.9 558. ]

data3 = data1 + 80%(data2)
data3 [  0.8   0.8   0.8 ... 558.8 741.8 620.8]


#### Discussion
- time mesurements are betters with simple addition than when we apply percentages

### Class and method for loading audio file by  file list


In [6]:
class load_bythread(Thread):
    """Thread to load audio file """
    wavefile = None
    data = None

    def __init__(self, wavefile):
        Thread.__init__(self)
        self.wavefile = wavefile

    def run(self):
        """Code du thread."""
        fs, self.data = wavfile.read(self.wavefile)

    def join(self):
        Thread.join(self)
        return self.data
    
def load_audiofile_list(pathfiles, by_thread=True, traceon=False):
    """ load audio file list
    input :
        pathfiles wave path files 
        two loading mode is proposed : 
        by_thread = True with multithreading
        by_thread = False without multithreading
        traceon : to display performance
    ouput :
        DataFrame with loaded data from wave files
    """    
    if traceon:
        debut_t = datetime.datetime.now()
    
    wavefiles = glob.glob(pathfiles + "//*.wav")
    df = pd.DataFrame(wavefiles)
    df.columns = ['fullname']
    df['file_name'] = df['fullname'].apply(lambda x: Path(x).name)
    
    if not by_thread:
        df['data'] = df['fullname'].apply(lambda x: load_wav_dataonly(x))
    
    if by_thread:
        df['load_bythread'] = df['fullname'].apply(lambda x: load_bythread(x))
        df.apply(lambda x: x['load_bythread'].start(), axis=1)
        df['data'] = df.apply(lambda x: x['load_bythread'].join(), axis=1)
        
    if traceon:
        s, ms, t = mesure_delta(debut_t, datetime.datetime.now())
        print('Time for loadind', len(wavefiles), ',audio files with mode mutithread=',by_thread,':', t)

    return df


### Test loading wave audio file list operation

In [28]:
def test_load_audiofile_list(pathfiles=None):
    """ test loading wav audio file list """
    if pathfiles is None:
        #pathfiles = '/mnt/d/ecomdata/DataForGood_wazo/audio-annotator/split_data-20191021T201218Z-001/split_data'
        pathfiles = '/mnt/d/ecomdata/DataForGood_wazo/input/ff1010bird_wav/wav'
    df = load_audiofile_list(pathfiles, by_thread=False, traceon=True)
    df = load_audiofile_list(pathfiles, by_thread=True, traceon=True)

test_load_audiofile_list()

Time for loadind 7690 ,audio files with mode mutithread= False : 238s 352ms


Exception in thread Thread-1110:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1107:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1114:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1

Exception in thread Thread-1506:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1508:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1507:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1

Exception in thread Thread-1914:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-1916:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined
Exception in thread Thread-1915:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-19

Exception in thread Thread-2321:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-2322:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-2323:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-2

...

Exception in thread Thread-7443:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-7442:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-7444:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-7

Exception in thread Thread-7865:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-7866:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-7867:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-7

Exception in thread Thread-8301:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined


Exception in thread Thread-8302:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-8303:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-

Exception in thread Thread-8739:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-8741:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-8740:
Traceback (most recent call last):
  File "/home/christian/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-26-2eca76ee1d63>", line 12, in run
    fs, self.data = wavfile.read(self.wavefile)
NameError: name 'wavfile' is not defined

Exception in thread Thread-8

Time for loadind 7690 ,audio files with mode mutithread= True : 6s 626ms


### class for adding audio operation with file list

In [6]:
class addwav_bythread(Thread):
    """Thread to load audio file """
    data = None

    def __init__(self, data1, data2, p1=1, p2=1):
        Thread.__init__(self)
        self.data1 = data1
        self.data2 = data2
        self.p1 = p1
        self.p2 = p2
        
    def run(self):
        """Code du thread."""
        self.data = add_dataWav1todataWav2(self.data1, self.data2, self.p1, self.p2) 

    def join(self):
        Thread.join(self)
        return self.data